In [8]:
%pip install guidance
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import guidance
from dotenv import load_dotenv

load_dotenv()

# To get a different result every time, uncomment the following line to clear the cache
# guidance.llms.OpenAI.cache.clear()

guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

#### Guardrail the LLM output structure

In [11]:
program = guidance('''
This is a rhyme by {{author}}:

"Roses are red, the water is blue {{gen "rhyme"}}

Another rhyme by {{author}}:
"{{gen "rhyme"}}"
''')

quote = program(author="janik")

quote


Stop program 
This is a rhyme by janik :

"Roses are red, the water is blue 
The sky is so high, and so are you 
Your beauty is like a summer day 
Your smile is like a ray of light that lights my way" 

Another rhyme by janik :
" The sun is so bright, the stars so high 
Your love is like a sweet lullaby 
Your eyes sparkle like the morning dew 
Your heart is so pure, it's all I need to get me through" "

#### Restrict LLM output selection

In [7]:
options = ['Yes', 'No', 'Maybe']

program = guidance('''Is the following sentence offensive? 
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
                   
executed_program = program(example='your tacos taste can be improved', options=options)


Stop program Is the following sentence offensive? 
Sentence: {{example}} 
Answer: {{select "answer" options=options}}

#### If conditions & Hidden generation

In [8]:
options = ["yes", "no"]

program = guidance(
'''User: {{query}}

{{#block hidden=True~}}
Is the user query rude or offensive?
Answer: {{select "rude" options=options}}
{{~/block~}}

{{#if rude=="yes"}}Assistant: Please be polite{{~/if~}}
{{#if rude=="no"}}Assistant: {{gen "answer"}}{{~/if~}}
''')
executed_program = program(query='your tacos taste too salty', options=options)



Stop program User: {{query}} 

 {{#block hidden=True~}} 
Is the user query rude or offensive?
Answer: {{select "rude" options=options}} 
 {{~/block~}} 

 {{#if rude=="yes"}} Assistant: Please be polite {{~/if~}} 
 {{#if rude=="no"}} Assistant: {{gen "answer"}} {{~/if~}}

# Generate emails with if condition

In [9]:
priorities = ["low priority", "medium priority", "high priority"]

email = guidance('''    
{{#block hidden=True~}}
Here is the customer message we received: {{email}}
Please give it a priority score
priority score: {{select "priority" options=priorities}}
{{~/block~}}
        
{{#block hidden=True~}}
You are a world class customer support; Your goal is to generate an response based on the customer message;
Here is the customer message to respond: {{email}}
Generate an opening & one paragraph of response to the customer message based on priority score: {{priority}}:
{{gen 'email_response'}} 
{{~/block~}}

{{email_response}}
{{#if priority=='high priority'}}Would love to setup a call this/next week, here is the calendly link: https://calendly.com/jason-zhou{{/if}}
Best regards
Jason
''')

email_response = email(email='What features does webflow have but wix dont have?', priorities=priorities)


Stop program 
 {{#block hidden=True~}} 
Here is the customer message we received: {{email}} 
Please give it a priority score
priority score: {{select "priority" options=priorities}} 
 {{~/block~}} 
 
 {{#block hidden=True~}} 
You are a world class customer support; Your goal is to generate an response based on the customer message;
Here is the customer message to respond: {{email}} 
Generate an opening & one paragraph of response to the customer message based on priority score: {{priority}} :
 {{gen 'email_response'}} 
 {{~/block~}} 

 {{email_response}} 
 {{#if priority=='high priority'}} Would love to setup a call this/next week, here is the calendly link: https://calendly.com/jason-zhou {{/if}} 
Best regards
Jason

# Generate chart

In [10]:
import urllib.parse

def generate_chart(query):
    
    def parse_chart_link(chart_details):
        encoded_chart_details = urllib.parse.quote(chart_details, safe='')

        output = "![](https://quickchart.io/chart?c=" + encoded_chart_details + ")"

        return output
    
    examples = [
        {
            'input': "Make a chart of the 5 tallest mountains",
            'output': {"type":"bar","data":{"labels":["Mount Everest","K2","Kangchenjunga","Lhotse","Makalu"], "datasets":[{"label":"Height (m)","data":[8848,8611,8586,8516,8485]}]}}
        },
        {
            'input': "Create a pie chart showing the population of the world by continent",
            'output': {"type":"pie","data":{"labels":["Africa","Asia","Europe","North America","South America","Oceania"], "datasets":[{"label":"Population (millions)","data": [1235.5,4436.6,738.8,571.4,422.5,41.3]}]}}
        }
    ]

    guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

    chart = guidance(
    '''    
    {{#block hidden=True~}}
    You are a world class data analyst, You will generate chart output based on a natural language;

    {{~#each examples}}
    Q:{{this.input}}
    A:{{this.output}}
    ---
    {{~/each}}
    Q:{{query}}
    A:{{gen 'chart' temperature=0 max_tokens=500}}    
    {{/block~}}
    
    Hello here is the chart you want
    {{parse_chart_link chart}}
    ''')

    return chart(query=query, examples=examples, parse_chart_link=parse_chart_link)


In [11]:
chart = generate_chart("Create a pie chart showing the population of the world by continent")

chart

Stop program 
 {{#block hidden=True~}} 
 You are a world class data analyst, You will generate chart output based on a natural language;

 {{~#each examples}} 
 Q: {{this.input}} 
 A: {{this.output}} 
 ---
 {{~/each}} 
 Q: {{query}} 
 A: {{gen 'chart' temperature=0 max_tokens=500}} 
 {{/block~}} 
 
 Hello here is the chart you want
 {{parse_chart_link chart}}

# Embed text to image 

In [12]:
def generate_story(story_idea):
        
    story = guidance('''
{{#block hidden=True~}}
You are a world class story teller; Your goal is to generate a short tiny story less than 200 words based on a story idea;

Here is the story idea: {{story_idea}}
Story: {{gen 'story' temperature=0}}
{{/block~}}

{{#block hidden=True~}}
You are a world class AI artiest who are great at generating text to image prompts for the story; 
Your goal is to generate a good text to image prompt and put it in a url that can generate image from the prompt;

Story: You find yourself standing on the deck of a pirate ship in the middle of the ocean. You are checking if there are still people on the ship
Image url markdown: ![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20man%20standing%20on%20the%20deck%20of%20a%20pirate%20ship%20in%20the%20middle%20of%20the%20ocean)
                
Story: {{story}}
Image url markdown: {{gen 'image_url' temperature=0 max_tokens=500}})
{{~/block~}}
                     
Story: {{~story~}}
{{image_url}}
''')

    story = story(story_idea=story_idea)
    print(story)
    return story

In [13]:
generate_story("little girl dreaming")

Stop program 
 {{#block hidden=True~}} 
You are a world class story teller; Your goal is to generate a short tiny story less than 200 words based on a story idea;

Here is the story idea: {{story_idea}} 
Story: {{gen 'story' temperature=0}} 
 {{/block~}} 

 {{#block hidden=True~}} 
You are a world class AI artiest who are great at generating text to image prompts for the story; 
Your goal is to generate a good text to image prompt and put it in a url that can generate image from the prompt;

Story: You find yourself standing on the deck of a pirate ship in the middle of the ocean. You are checking if there are still people on the ship
Image url markdown: ![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20man%20standing%20on%20the%20deck%20of%20a%20pirate%20ship%20in%20the%20middle%20of%20the%20ocean)
 
Story: {{story}} 
Image url markdown: {{gen 'image_url' temperature=0 max_tokens=500}} )
 {{~/block~}} 
 
Story: {{~story~}} 
 {{image_url}}